## Áp dụng CNN cho bài toán MNIST




In [1]:
try:
  # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x  #gpu
except Exception:
    pass
import tensorflow as tf

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
import os
import tensorflow_datasets as tfds #https://www.tensorflow.org/datasets/catalog/overview

print(tf.__version__)

2.2.0


## Tải tập dữ liệu về máy

Download tập dữ liệu MNIST và tải lên xài trong chương trình hiện tại từ [đây](https://www.tensorflow.org/datasets). Hàm này trả ra tập huấn luyện ở định dạng `tf.data`.

Thiết lập `with_info` thành `True` bao bao gồm metadata cho toàn bộ tập dữ liệu, và nó được lưu trong `info`. Metadata bao gồm số lượng mẫu trong tập huấn luyện (train set) và tập kiểm thử (test set). 


In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


## Định nghĩa chiến lược phân tán (distribution strategy)

API `tf.distribute.Strategy` cho trừu tượng hoá quá trình phân tán tác vụ huấn luyện (distribute training) ra nhiều đơn vị xử lý đồng thời. Người dùng thông qua đó có thể phân tán tác vụ huấn luyện với mô hình (model) và code huấn luyện sẵn có mà chỉ cần một vài thay đổi nhỏ.

Ở đây ta xài `tf.distribute.MirroredStrategy`, nó sẽ tự nhân bản và đồng bộ tác vụ huấn luyện trên nhiều GPU (của cùng một máy). Về bản chất, nó sẽ sao chép tất cả biến số của mô hình (models's variables) lên mỗi vi xử lý (processor). Tiếp đó, nó xài [all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) để tổng hợp các gradient từ tất cả vi xử lý (processor) và áp dụng các giá trị tổng hợp được này cho tất cả bản sao của mô hình.

Trong bài này ta xài `MirroredStategy`. Đây là 1 trong số các chiến lược phân tán (distribution strategy) sẵn có trong bộ lõi của TensorFlow. Xem thêm về các chiến lược phân tán khác ở [đây](https://www.tensorflow.org/guide/distributed_training)

Tạo đối tượng (object) `MirroredStrategy`. Nó sẽ lãnh việc `xử lý phân tán` (distribution), và cung cấp trình quản lý ngữ cảnh (context manager)(`tf.distribute.MirroredStrategy.scope`) để xây dựng mô hình huấn luyện.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [ ]:
tf.test.is_gpu_available()

False

## Chuẩn bị chuỗi đầu vào theo lô (input pipeline)

Trong quá trình huấn luyện mô hình sử dụng nhiều GPU đồng thời. Để hiệu quả, ta sẽ tăng kích thước lô (batch size). Nhìn chung, ta sẽ thiết lập kích thước lô vừa vặn với bộ nhớ của GPU luôn, và tinh chỉnh (tune) tỉ lệ học (learning rate) theo cho phù hợp.

In [ ]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Giá trị của mỗi *Pixel* nằm trong khoảng 0-255, [cần phải được chuẩn hoá về khoảng 0-1](https://en.wikipedia.org/wiki/Feature_scaling). Ta định nghĩa hàm căng chỉnh (scale) giá trị như sau:

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  # TODO: scaling value of each pixel here:
  image = image/255
  return image, label

In [ ]:
print(mnist_train)

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>


Áp dụng hàm này cho cả tập huấn luyện và tập kiểm thử, xáo trộn tập dữ liệu huấn luyện và [phân lô (batch) để huấn luyện](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch). Chú ý: ta cũng lưu trữ cache của dữ liệu huấn luyện để cải thiện hiệu suất.


In [ ]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [ ]:
print(train_dataset)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


## Thiết kế mô hình

Thiết kế và phiên dịch mô hình Keras trong ngữ cảnh (context) `strategy.scope`.

In [ ]:
# TODO: set suitable value to below variables
HIDDEN_LAYER_SIZE = 128 #hyperparameter
OUTPUT_SIZE = 10

with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(), #2,2

      tf.keras.layers.Flatten(),
      
      # TODO: fill suitable activations
      tf.keras.layers.Dense(HIDDEN_LAYER_SIZE, activation='relu'),
      tf.keras.layers.Dense(OUTPUT_SIZE, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', #https://keras.io/losses/
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
# TODO: set suitable value to below variables
hidden_layer_size = 128
output_size = 10

with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      
      tf.keras.layers.MaxPooling2D(),

      tf.keras.layers.Flatten(),
      
      # TODO: fill suitable activations
      tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(output_size, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

## Định nghĩa callback



Ta dùng Callback ở đây để:

*   *Model Checkpoint*: lưu trữ (giá trị) mô hình sau mỗi lượt huấn luyện (epoch).

In [ ]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
]

## Huấn luyện và đánh giá:

Huấn luyện mô hình, gọi hàm `fit` trên biến model.


In [ ]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
938/938 [==============================] - 55s 59ms/step - loss: 0.1664 - accuracy: 0.9502
Epoch 2/12
922/938 [============================>.] - ETA: 0s - loss: 0.0502 - accuracy: 0.9845

KeyboardInterrupt: 

As you can see below, the checkpoints are getting saved.

In [ ]:
# check the checkpoint directory
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00002
ckpt_10.data-00000-of-00002  ckpt_4.data-00001-of-00002
ckpt_10.data-00001-of-00002  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00002
ckpt_11.data-00000-of-00002  ckpt_5.data-00001-of-00002
ckpt_11.data-00001-of-00002  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00002
ckpt_12.data-00000-of-00002  ckpt_6.data-00001-of-00002
ckpt_12.data-00001-of-00002  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00002
ckpt_1.data-00000-of-00002   ckpt_7.data-00001-of-00002
ckpt_1.data-00001-of-00002   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00002
ckpt_2.data-00000-of-00002   ckpt_8.data-00001-of-00002
ckpt_2.data-00001-of-00002   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00002
ckpt_3.data-00000-of-00002   ckpt_9.data-00001-of-00002
ckpt_3.data-00001-of-00002   ckpt_9.index
ckpt_3.index


Để xem mô hình ta đã huấn luyện tốt thế nào, ta tải checkpoint cuối cùng lên và gọi hàm `evaluate` trên tập kiểm thử.

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


    157/Unknown - 2s 13ms/step - loss: 0.0425 - accuracy: 0.9901Eval loss: 0.042545092066180665, Eval Accuracy: 0.9901000261306763


## Xuất và lưu mô hình huấn luyện

In [ ]:
path = 'saved_model/'

In [ ]:
model.save(path, save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Tải mô hình bên ngoài `strategy.scope`.

In [ ]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.fit (data)

unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

NameError: ignored

Tải mô hình trong `strategy.scope`.

In [ ]:
with strategy.scope():
    replicated_model = tf.keras.models.load_model(path)
    replicated_model.compile(loss='sparse_categorical_crossentropy',
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=['accuracy'])

    eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
    print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))